In [1]:
import pandas as pd
import pickle
with open('../population_dict.pickle', 'rb') as f:
    population_dict = pickle.load(f)

## VK data

In [9]:
vk=pd.read_csv("../vk_dataset.csv")

In [15]:
vk.head()

,phone,sex,region,age,count,population,prob
0,Android high price,man,Москва,0.0,9000,10381222,0.000867
1,Android high price,woman,Москва,0.0,7900,10381222,0.000761
2,Android high price,woman,Москва,1.0,13000,10381222,0.001252
3,Android high price,man,Москва,1.0,28000,10381222,0.002697
4,Android high price,man,Москва,2.0,34000,10381222,0.003275


In [11]:
vk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   phone   1512 non-null   object 
 1   sex     1512 non-null   object 
 2   region  1512 non-null   object 
 3   age     1512 non-null   float64
 4   count   1512 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 59.2+ KB


In [12]:
vk["population"]=vk.region.apply(lambda x: population_dict.get(x, -1))

In [14]:
vk["prob"]=vk.apply(lambda x: int(x["count"])/int(x["population"]) , axis=1)

In [16]:
vk.to_csv("../vk_dataset_new.csv", index=False)

## Prepare test data

In [44]:

    


def preprocessing_5_test(path):
    train = pd.read_csv(path)
    
    train['shift']=train['shift'].fillna(0)
    train['os']=train['shift'].fillna("ios")
    train['osv']=train['shift'].fillna("10.0")
    
    
    train["data_type"]=train.apply(lambda x: 1 if (("I" in str(x["os"])) or ("A" in str(x["os"]))) else 0 , axis=1)
    # train = train.drop(train[train['os'] == "nan"].index)
    train["os"]=train.apply(lambda x: str(x["os"]).lower().strip() , axis=1)

    # train=train.dropna()
    # train.loc[train['Segment'] == 1, 'Segment'] = 2
    
    # train = pd.read_csv('../train.csv')
    # train=train.dropna()
    train['shift']=train.apply(lambda x: int(float(str(x['shift']).replace("MSK",""))) if str(x['shift']).replace("MSK","")!="" else 0 , axis=1)
    train["created"] = pd.to_datetime(train["created"], format='%Y-%m-%d %H:%M:%S', errors='ignore')
    train["new_created"]=train.apply(lambda x: x["created"]+pd.Timedelta(hours=x["shift"]) , axis=1)
    
    train['created_year'] = train["created"].dt.year
    train['created_month'] = train["created"].dt.month
    train['created_day'] = train["created"].dt.day
    train['created_hour'] = train["created"].dt.hour
    train['created_minute'] = train["created"].dt.minute
    train['created_second'] = train["created"].dt.second
    
    train['created_year_new'] = train["new_created"].dt.year
    train['created_month_new'] = train["new_created"].dt.month
    train['created_day_new'] = train["new_created"].dt.day
    train['created_hour_new'] = train["new_created"].dt.hour
    train['created_minute_new'] = train["new_created"].dt.minute
    train['created_second_new'] = train["new_created"].dt.second
    
    train['day_of_week'] = train.apply(lambda x: x["created"].weekday(), axis=1)
    train['day_of_week_new'] = train.apply(lambda x: x["new_created"].weekday(), axis=1)
    
    train['city_population'] = train.city.apply(lambda x: population_dict.get(x, -1))

    del train["created"]
    del train["new_created"]
    
    train["bundle"]=train.apply(lambda x: str(x["bundle"]).lower().strip() , axis=1)

    # train['Segment']=train.apply(lambda x: int(x["Segment"]), axis=1)
    # train = train.drop_duplicates()
    train['city_population'] = train['city_population'].fillna(-1)
    for col in train.select_dtypes(include=['object']).columns:
        train[col] = train[col].astype('category')
    
    for col in train.select_dtypes(include=['category']).columns:
        train[col] = train[col].cat.codes
    return train

In [45]:
test=preprocessing_5_test("../test.csv")

In [78]:
test.to_csv("../test_post.csv", index=False)

## Run CatBoost

In [2]:
test=pd.read_csv("../test_post.csv")

In [79]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.load_model("../notebooks/catboost_5_all")

In [80]:
preds_class = model.predict(test)
preds_proba = model.predict_proba(test)

In [85]:
dict_res={"id":[], "predict":[], "probability":[]}
for i, cl, prob in zip(range(test.shape[0]),preds_class, preds_proba):
    dict_res["id"].append(i)
    dict_res["predict"].append(cl[0])
    dict_res["probability"].append(prob)
            

In [89]:
a=pd.DataFrame.from_dict(dict_res)
a.to_csv("../test_predict.csv", index=False)

In [ ]:
# Segment
# 5    17187506
# 3    14187054
# 4    11142080
# 2     1416245
# 1      921631

In [91]:
a["predict"].value_counts()

5    5730414
3    5327420
4     135804
2      19262
1        729
Name: predict, dtype: int64

## Run RandomForest

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import ensemble, metrics
import xgboost as xgb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [4]:
model = ensemble.RandomForestClassifier()

In [5]:
import pickle
with open('RFC_n_est10_deep50_all', 'rb') as f:
        model=pickle.load(f)

In [6]:
t_=test.head()

In [ ]:
Index(['gamecategory', 'subgamecategory', 'bundle', 'shift', 'oblast', 'city',
       'os', 'osv', 'created_year', 'created_month', 'created_day',
       'created_hour', 'created_minute', 'created_second', 'created_year_new',
       'created_month_new', 'created_day_new', 'created_hour_new',
       'created_minute_new', 'created_second_new', 'day_of_week',
       'day_of_week_new', 'city_population'],
      dtype='object')

In [9]:
t_.columns

Index(['gamecategory', 'subgamecategory', 'bundle', 'shift', 'oblast', 'city',
       'os', 'osv', 'data_type', 'created_year', 'created_month',
       'created_day', 'created_hour', 'created_minute', 'created_second',
       'created_year_new', 'created_month_new', 'created_day_new',
       'created_hour_new', 'created_minute_new', 'created_second_new',
       'day_of_week', 'day_of_week_new', 'city_population'],
      dtype='object')

In [8]:
t_.head()

,gamecategory,subgamecategory,bundle,shift,oblast,city,os,osv,data_type,created_year,...,created_second,created_year_new,created_month_new,created_day_new,created_hour_new,created_minute_new,created_second_new,day_of_week,day_of_week_new,city_population
0,0,40,11613,2,68,589,3,3,0,2021,...,33,2021,9,19,19,31,33,6,6,1349772.0
1,-1,-1,16602,2,88,1470,3,3,0,2021,...,17,2021,8,3,19,13,17,1,1,94212.0
2,-1,-1,2537,0,47,1279,1,1,0,2021,...,0,2021,9,17,15,54,0,4,4,10381222.0
3,-1,-1,17381,0,35,983,1,1,0,2021,...,59,2021,7,5,23,34,59,0,0,649851.0
4,0,23,37557,0,73,1312,1,1,0,2021,...,21,2021,7,8,15,15,21,3,3,509870.0


In [7]:
preds_class = model.predict(t_)

ValueError: X has 24 features, but RandomForestClassifier is expecting 23 features as input.

In [ ]:
preds_class

In [ ]:
dict_res={"id":[], "predict":[]}
for i, cl, prob in zip(range(test.shape[0]),preds_class):
    dict_res["id"].append(i)
    dict_res["predict"].append(cl[0])

In [ ]:
a["predict"].value_counts()

In [ ]:
a=pd.DataFrame.from_dict(dict_res)
a.to_csv("../test_predict.csv", index=False)